In [ ]:
!pip install fastapi nest_asyncio pyngrok uvicorn

In [ ]:
!pip install langchain
!pip install ctransformers
!pip install langchain_community

In [ ]:
import csv
from langchain import PromptTemplate, LLMChain
from langchain_community.llms import CTransformers

# Define the updated prompt template
template = """
You are a teacher preparing questions for a quiz. Based on the given word, please generate one multiple-choice question (MCQ) with 4 options. The options should be labeled as "A)", "B)", "C)", and "D)". Also, provide the correct answer, and ensure the answer exactly matches one of the options, including the label.

Example:

Question: What is the primary purpose of a firewall?
A) To enhance system speed
B) To block unauthorized access
C) To monitor CPU temperature
D) To improve battery life
Answer: B) To block unauthorized access

Please ensure the question is clear, the options are plausible, and the answer matches one of the options exactly.

WORD:
{word}

QUESTION:
"""

# Initialize the language model
llm = CTransformers(model='/content/drive/MyDrive/Dissertation/mistral-7b-instruct-v0.2.Q5_K_M.gguf', model_type='mistral')

# Create the prompt template
prompt = PromptTemplate(template=template, input_variables=['word'])

# Initialize the LLMChain
llm_chain = LLMChain(prompt=prompt, llm=llm)

# List of words for generating questions
words_list = [
    'Cybersecurity', 'Encryption', 'Firewall', 'Malware',
    'Phishing', 'Ransomware', 'Authentication', 'Antivirus',
    'Spyware', 'Data Breach'
]

# Prepare to save to CSV
csv_file = '/content/drive/MyDrive/Dissertation/web/assets/quizdata.csv'
fieldnames = ['question', 'option1', 'option2', 'option3', 'option4', 'answer']

with open(csv_file, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()

    # Generate unique questions and answers for each word
    for i, word in enumerate(words_list):
        print(f"Generating question for: {word}")
        question_and_answer = llm_chain.run(word).strip()

        # Split the generated output into question, options, and answer
        lines = question_and_answer.splitlines()
        question_line = lines[0].replace("Question:", "").strip()
        option_a = lines[1].strip()
        option_b = lines[2].strip()
        option_c = lines[3].strip()
        option_d = lines[4].strip()
        answer_text = lines[5].replace("Answer:", "").strip()

        # Write to CSV
        writer.writerow({
            'question': question_line,
            'option1': option_a,
            'option2': option_b,
            'option3': option_c,
            'option4': option_d,
            'answer': answer_text
        })

print(f"Questions saved to {csv_file}")


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


Generating question for: Cybersecurity
Generating question for: Encryption
Generating question for: Firewall
Generating question for: Malware
Generating question for: Phishing
Generating question for: Ransomware
Generating question for: Authentication
Generating question for: Antivirus
Generating question for: Spyware
Generating question for: Data Breach
Questions saved to /content/drive/MyDrive/Dissertation/web/assets/quizdata.csv


In [30]:
!npm install -g localtunnel



changed 22 packages, and audited 23 packages in 1s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [31]:
#%%writefile /content/drive/MyDrive/Dissertation/web/app.py
# app.py

import nest_asyncio
import uvicorn
from fastapi import FastAPI
from fastapi.responses import HTMLResponse
from fastapi.staticfiles import StaticFiles
from pathlib import Path
import subprocess
import threading

# Apply nest_asyncio to work in Jupyter
nest_asyncio.apply()

app = FastAPI()

# Define the path to your HTML files and assets
html_dir = Path("/content/drive/MyDrive/Dissertation/web")
assets_dir = html_dir / "assets"

# Serve static files (e.g., images, CSS, JS)
app.mount("/assets", StaticFiles(directory=assets_dir), name="assets")

# Route for serving HTML files
@app.get("/", response_class=HTMLResponse)
async def get_index():
    with open(html_dir / "index.html") as f:
        return HTMLResponse(content=f.read())

# Serve figma5.html file
@app.get("/figma5", response_class=HTMLResponse)
async def get_figma5():
    with open(html_dir / "figma5.html") as f:
        return HTMLResponse(content=f.read())

# Serve feedback.html file
@app.get("/feedback", response_class=HTMLResponse)
async def get_feedback():
    with open(html_dir / "feedback.html") as f:
        return HTMLResponse(content=f.read())

# Function to start LocalTunnel in a separate thread and print only the URL and password first
def start_local_tunnel(port, subdomain, password):
    # Command to start LocalTunnel with a specific subdomain
    tunnel_command = ["lt", "--port", str(port), "--subdomain", subdomain]

    # Start the tunnel process
    tunnel_process = subprocess.Popen(tunnel_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Capture and print the URL
    for line in iter(tunnel_process.stdout.readline, b''):
        decoded_line = line.decode().strip()
        if 'https://' in decoded_line:
            print(f"LocalTunnel URL: {decoded_line}")

            # Authenticate using curl if a password is needed
            if password:
                auth_url = f"https://{subdomain}.loca.lt/{password}"
                print(f"Authenticating with password: {password}")
                curl_command = ["curl", auth_url]
                subprocess.run(curl_command)
            break

    # Print remaining logs (stdout and stderr)
    while True:
        output = tunnel_process.stdout.readline()
        if output == b'' and tunnel_process.poll() is not None:
            break
        if output:
            print(output.decode().strip())

    for line in iter(tunnel_process.stderr.readline, b''):
        print(line.decode().strip())

# Run the FastAPI server and LocalTunnel
port = 8000
subdomain = "mytunnel"  # Replace with your desired subdomain
password = "yourpassword"  # Replace with your password

# Start LocalTunnel in a separate thread with password authentication
tunnel_thread = threading.Thread(target=start_local_tunnel, args=(port, subdomain, password), daemon=True)
tunnel_thread.start()

# Run the FastAPI server
uvicorn.run(app, host="0.0.0.0", port=port)


INFO:     Started server process [270]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


LocalTunnel URL: your url is: https://mytunnel.loca.lt
Authenticating with password: yourpassword
INFO:     35.186.179.214:0 - "GET /yourpassword HTTP/1.1" 404 Not Found
INFO:     31.205.104.74:0 - "GET / HTTP/1.1" 200 OK
INFO:     31.205.104.74:0 - "GET /figma5 HTTP/1.1" 200 OK
INFO:     31.205.104.74:0 - "GET / HTTP/1.1" 200 OK
INFO:     31.205.104.74:0 - "GET /feedback HTTP/1.1" 200 OK
INFO:     31.205.104.74:0 - "GET / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [270]


In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
